# 对yolov5中detect.py重新封装


detect中代码的所有功能都被写进了一个run函数里，显得代码十分臃肿，优化扩展可能性低，且有些功能并不需要，所有将其封装成多个功能的函数，各司其职，能增加代码的可读性。

### 一、首先要读懂代码，理解各个部分的内容
在b站和CSDN上查找资料学习，detect中的run函数代码可以分八个功能：生成保存目录，加载模型，加载数据，预处理，推理，后处理结果及可视化。其中主要的部分就是预处理到可视化，下面我主要解释下我对这部分代码的理解。
    
    

#### (1)预处理，推理
由于输入的图片规格各异，所以先将其进行处理为统一的格式：
                    im = torch.from_numpy(im).to(model.device)   #将numpy转为torch张量
                    im = im.half() if model.fp16 else im.float() # 确定图像的精度
                    im /= 255                                    # 将图像的像素值归一化


然后开始推理，此时推理出的框未经过筛选会有很多噪声，所以由非极大值抑制来筛选：
                    pred = model(im, augment=augment, visualize=visualize)#此时会得到很多的框
                    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)#筛选


#### (2)后处理及可视化
由Annotator这个定义好的类从预测结果中得值画框。以及一些需要映射回原图的过程

### 二、冗长原因
总体看来，其实冗长的主要原因是其中的run_inference的部分。将其拆开成两个函数，后处理及推理会简洁很多，且能更方便的在后处理部分添加其他所需要的功能。
重新封装后，我把代码分成这几个部分：init初始化、数据模型加载、后处理部分、推理部分。

以及其中有很多各样的判断，是否保存labels，是否直接显示什么的，也让代码看的很复杂。所以可在这些方面化简


以及更改成输出想要的日志或内容，我设置了一个判断是否检测到某物体的输出